<b>Audio and Music Processing Lab - Module 2</b><br>Gonzalo Nieto<br>Morgan Buisson<br>9.03.2021
## AMPLab2 - Ethnomusicology research
### Semantic melodic comparison
The aim of this notebook is to compute the melodic distances between poems inside the same similarity or disimilarity groups.
It uses data processed with ```TopicExtraction.ipynb``` and ```Melodic_analysis.ipynb``` notebooks. Note that there might be paths that need to be changed. 

In [1]:
from music21 import *
import json
import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

Computes L1 distance between two dictionaries of intervals.

In [2]:
def compare_melodies(d1, d2):
    total = 0
    for i in list(d1.keys()):
        if i in list(d2.keys()):
            total += abs(d1[i]-d2[i])
        else:
            total += d1[i]
    for j in list(d2.keys()):
        if j not in list(d1.keys()):
            total+=d2[j]
    return total

Normalizes the dictionaries computed in the ```Melodic_analysis.ipynb```, so that all poems can be comparable.

In [3]:
def normalize(intervals_dict):
    for poem in intervals_dict:
        total = sum([intervals_dict[poem][i] for i in intervals_dict[poem]])
        for itv in intervals_dict[poem]:
            intervals_dict[poem][itv] = intervals_dict[poem][itv]/total
    return intervals_dict

Compare in melodic terms the 6 semantically similar poems of each group (given by each entry of the semantic_similarity_file dictionary). The goal is to calculate the L1 difference between the interval counts of the key poem and each of the other 5 poems of its similarity (or disimilarity) group and then calculate the mean. The result is a dictionary which keys are poems (tuples with recording mbid and poem identifier) and values are the averages of the L1 distance of the group.

In [4]:
def melodic_similarity(semantic_similarity_file):
    with open(semantic_similarity_file, "rb") as f:
            similarity_semantic = pickle.load(f)

    diff_mean = {}
    for tuples in tqdm(similarity_semantic):
        if tuples[0] != '.DS_Store':
            with open(f'./intervals/{tuples[0]}_intervals.json', "rb") as f: # open the json with the melodic analysis
                recording_intervals = normalize(json.load(f))
            key_melody = recording_intervals[tuples[1]] # analysis of the 'key' poem

            temp_list = []
            for i in range(len(similarity_semantic[tuples])):
                mbid = similarity_semantic[tuples][i][0]
                poem = similarity_semantic[tuples][i][1]
                if mbid != '.DS_Store':
                    with open(f'./intervals/{mbid}_intervals.json', "rb") as f:
                        recording_intervals = normalize(json.load(f))
                    melody_x = recording_intervals[poem] # analysis of one similar poem of the group of five
                    diff = compare_melodies(key_melody, melody_x)
                    print(f'- Diffence between {tuples} (key) and ({mbid}, {poem}): {diff}')
                    temp_list.append(diff)
            diff_mean[tuples] = np.mean(temp_list)
    return diff_mean

In [6]:
semantic_similarity_file = './results/similarity_original.txt'
difference_dict = melodic_similarity(semantic_similarity_file)
f = open(f'{semantic_similarity_file[:-4]}_melodic_difference.txt',"wb")
pickle.dump(difference_dict, f) 
f.close() 

c-236110dde3c0', 'Ahsanta') (key) and (3e5a82a2-d806-45cc-876e-6fa8a2b5a61d, Abshir laqad): 0.32771428571428574

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (bb7adfee-f63d-4004-a457-b31ae784a0bd, Aya fadiha): 1.036451127819549

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (1ffa4dee-b08e-4332-a2ab-9843c13c3dfa, Wa man takun birasuli Allahi): 0.7592727272727272

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (e79c1615-7172-43a4-acdf-e9475c224edf, Yufuni): 0.17461093150890913

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (e79c1615-7172-43a4-acdf-e9475c224edf, Ya laylatan): 0.14086809767417327

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (3e5a82a2-d806-45cc-876e-6fa8a2b5a61d, Abshir laqad): 0.4362695608278647

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awd

In [7]:
semantic_similarity_file = './results/similarity_transliterated.txt'
difference_dict = melodic_similarity(semantic_similarity_file)
f = open(f'{semantic_similarity_file[:-4]}_melodic_difference.txt',"wb")
pickle.dump(difference_dict, f) 
f.close() 

 and (3e39495a-443d-483d-93d7-cdf96248c374, Ahsanta ya laylu): 0.8640829105041564

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Andante') (key) and (e79c1615-7172-43a4-acdf-e9475c224edf, Ya laylatan): 0.7591702504427017

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (e79c1615-7172-43a4-acdf-e9475c224edf, Ahsanta ya laylu): 0.8383694267515924

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (e4e3fc3c-8d57-4123-b14c-236110dde3c0, Andante): 0.7277288135593218

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (3e39495a-443d-483d-93d7-cdf96248c374, Ahsanta ya laylu): 0.8383694267515924

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (e79c1615-7172-43a4-acdf-e9475c224edf, Ya laylatan): 0.667820895522388

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (e0ce0420-165c-4339-8556-6d75a0b733c7, Allaho ya`la

In [10]:
semantic_similarity_file = './results/dissimilarity_original.txt'
difference_dict = melodic_similarity(semantic_similarity_file)
f = open(f'{semantic_similarity_file[:-4]}_melodic_difference.txt',"wb")
pickle.dump(difference_dict, f) 
f.close() 

e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (6fe7108c-4e4f-457b-a363-ccf505bdee9a, Qad bacharat): 0.19701901140684414

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (0d1d3cff-e68e-403d-bb3f-caca302bf5ab, Irham): 0.3357037037037037

- Diffence between ('e4e3fc3c-8d57-4123-b14c-236110dde3c0', 'Ahsanta') (key) and (b787f4e2-dce1-4bfe-89b3-193cbaa5c90d, Tala ightirabi): 0.29409900990099

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (62acb9e5-e311-40ab-9c5c-866d071b5c93, Bidimam al-hawa): 0.31169662518564034

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (2cf367de-583b-4bd6-83f3-fc036fe19fab, Ya ahla taybata li): 0.2551448498564074

- Diffence between ('99004770-1152-4fec-802a-a0014e9b9930', 'Binourika awdahta') (key) and (e0ce0420-165c-4339-8556-6d75a0b733c7, Allaho ya`lamo): 0.3597990432922286

- Diffence between ('99004770-1152-4fec-802a-a0014e9b99

In [11]:
semantic_similarity_file = './results/dissimilarity_transliterated.txt'
difference_dict = melodic_similarity(semantic_similarity_file)
f = open(f'{semantic_similarity_file[:-4]}_melodic_difference.txt',"wb")
pickle.dump(difference_dict, f) 
f.close() 

43a4-acdf-e9475c224edf, Ya laylatan): 0.529258469556977

- Diffence between ('59fcb1d1-61f4-4cdc-9f7f-56757bf2400e', 'Ghazalun sama') (key) and (0d1d3cff-e68e-403d-bb3f-caca302bf5ab, Addunia hullat): 0.6918407128933445

- Diffence between ('59fcb1d1-61f4-4cdc-9f7f-56757bf2400e', 'Ghazalun sama') (key) and (56da9cc8-dbdf-4123-9f05-d866b8380d93, Ra´ayto alhilala): 0.5299869802487605

- Diffence between ('59fcb1d1-61f4-4cdc-9f7f-56757bf2400e', 'Ghazalun sama') (key) and (2cf367de-583b-4bd6-83f3-fc036fe19fab, Ya ahla taybata li): 0.6748706973426074

- Diffence between ('59fcb1d1-61f4-4cdc-9f7f-56757bf2400e', 'Ghazalun sama') (key) and (13a8d4b9-8dbf-40c0-98df-db64791e43a9, Wa za`iri): 0.48815274902231426

- Diffence between ('3e5a82a2-d806-45cc-876e-6fa8a2b5a61d', 'Ja´a al-bashiru') (key) and (d5cc773e-9d31-42e8-b9de-91374fe7153b, Ma kanzi wa itimadi): 0.21084217506631292

- Diffence between ('3e5a82a2-d806-45cc-876e-6fa8a2b5a61d', 'Ja´a al-bashiru') (key) and (860a5ee6-6f83-42b1-a277-8eba